The **discounted payback period** is one of several capital budgeting methods used to evaluate capital investments. It gives the number of years required to recover the original investment in a project. The discounted payback period method takes into account the *time value of money* by discounting the future cash flows at a desired rate of return.

Let us see how we can calculate the discounted payback period of a project using NumPy and Pandas.

Consider a project with an initial outlay of \$750 million with expected cash inflows of \$175 million per year for seven years. The investor, desiring a 10% rate of return, is interested in knowing the number of years it will take to recover his/her original investment of \$750 million.

In [1]:
import numpy as np
from pandas import Series, DataFrame

Let's create a variable for the 10% rate of return and a list for the \$750 million cash outflow followed by seven $175 million cash inflows. For convenience, the outflow and inflows are marked as negative and positive, respectively.

In [2]:
# Desired rate of return is 10%
rate = 10 / 100

# Amounts in millions
cash_flows = [-750] + [175] * 7
rate, cash_flows

(0.1, [-750, 175, 175, 175, 175, 175, 175, 175])

Convert the cash flows array to a DataFrame. Since the cash flows are undiscounted, let’s name the column *UndiscountedCashFlows*

In [3]:
cf_df = DataFrame(cash_flows, columns=['UndiscountedCashFlows'])
cf_df.index.name = 'Year'
cf_df

,UndiscountedCashFlows
Year,
0,-750
1,175
2,175
3,175
4,175
5,175
6,175
7,175


Now, let’s calculate the present values of each of the future cash flows using numpy’s *npv()* method. Add these values as a new column *DiscountedCashFlows* to the DataFrame.

In [4]:
cf_df['DiscountedCashFlows'] = np.pv(rate=rate, pmt=0, nper=cf_df.index, fv=-cf_df['UndiscountedCashFlows'])
cf_df

,UndiscountedCashFlows,DiscountedCashFlows
Year,,
0,-750,-750.000000
1,175,159.090909
2,175,144.628099
3,175,131.480090
4,175,119.527355
5,175,108.661232
6,175,98.782938
7,175,89.802671


We need to know the cumulative sum of the discounted cash flows to calculate the payback period. Let’s add another column *CumulativeDiscountedCashFlows* to the DataFrame to store this cumulative sum.

In [5]:
cf_df['CumulativeDiscountedCashFlows'] = np.cumsum(cf_df['DiscountedCashFlows'])
cf_df

,UndiscountedCashFlows,DiscountedCashFlows,CumulativeDiscountedCashFlows
Year,,,
0,-750,-750.000000,-750.000000
1,175,159.090909,-590.909091
2,175,144.628099,-446.280992
3,175,131.480090,-314.800902
4,175,119.527355,-195.273547
5,175,108.661232,-86.612315
6,175,98.782938,12.170622
7,175,89.802671,101.973293


As you can see in the output above, at the end of Year 6, the value of Cumulative Discounted Cash Flows becomes positive for the first time. This means that project has recovered its original investment and has now entered the realm of profitability.
Therefore, the discounted payback period is between five and six years. Let’s see at what point in the sixth year the project becomes profitable. For this, we will divide the cumulative sum at the end of the fifth year by the sixth year’s discounted cash flow.

In [6]:
final_full_year = cf_df[cf_df.CumulativeDiscountedCashFlows < 0].index.values.max()
final_full_year

5

In [7]:
fractional_yr = -cf_df.CumulativeDiscountedCashFlows[final_full_year ] / cf_df.DiscountedCashFlows[final_full_year + 1]
fractional_yr

0.87679428571428863

In [8]:
payback_period = final_full_year + fractional_yr
payback_period

5.876794285714289

Let’s save the above code as a function and call it on different projects.

In [9]:
def discounted_payback_period(rate, cash_flows=list()):    
    cf_df = DataFrame(cash_flows, columns=['UndiscountedCashFlows'])
    cf_df.index.name = 'Year'
    cf_df['DiscountedCashFlows'] = np.pv(rate=rate, pmt=0, nper=cf_df.index, fv=-cf_df['UndiscountedCashFlows'])
    cf_df['CumulativeDiscountedCashFlows'] = np.cumsum(cf_df['DiscountedCashFlows'])
    final_full_year = cf_df[cf_df.CumulativeDiscountedCashFlows < 0].index.values.max()
    fractional_yr = -cf_df.CumulativeDiscountedCashFlows[final_full_year ] / cf_df.DiscountedCashFlows[final_full_year + 1]
    payback_period = final_full_year + fractional_yr
    return payback_period

# Example 1:

Calculate the discounted payback period of a project which requires an initial investment of \$5,000 million and results in cash flows of \$1,500 million each for the next 5 years. The expected rate of return is 10%.

In [10]:
rate = 0.1
cash_flows = [-5000] + [1500] * 5
print('The discounted payback period for this project is {:.2f} years.'.format(discounted_payback_period(rate, cash_flows)))

The discounted payback period for this project is 4.26 years.


# Example 2:

Consider a project with the following cash flows. Calculate its discounted payback period for a rate of return of 8%.<br>
**Year 0:** Initial investment of \$50,000,<br>
**Years 1 and 2:** Cash inflows of \$15,000 each,<br>
**Year 3:** Cash inflow of \$20,000,<br>
**Year 4:** Cash inflow of \$10,000, and<br>
**Year 5:** Cash inflow of \$5,000<br>

In [11]:
rate = 0.08
cash_flows = [-50000, 15000, 15000, 20000, 10000, 5000]
print('The discounted payback period for this project is {:.2f} years.'.format(discounted_payback_period(rate, cash_flows)))

The discounted payback period for this project is 4.01 years.
